# 사전 훈련된 워드 임베딩(Pre-trained Word Embedding

케라스 임베딩 층(Keras Embedding Layer)

문장의 긍, 부정을 판단하는 감성 분류 모델 예시
긍정적인 문장은 레이블1, 부정인 문장은 레이블0 으로 하면

In [4]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 + 1
print('단어 집합', vocab_size)

단어 집합 16


정수 인코딩 진행

In [5]:
x_encoded = tokenizer.texts_to_sequences(sentences)
print(x_encoded)

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


가장 길이가 긴 문장 길이

In [8]:
max_len = max(len(l) for l in x_encoded)
print(max_len)

4


최대 길이로 모든 샘플에 대해서 패딩 진행

In [15]:
x_train = pad_sequences(x_encoded, maxlen=max_len, padding='post')
print(x_train)
print(y_train)
y_train = np.array(y_train)
print(y_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]
[1 0 0 1 1 0 1]
[1 0 0 1 1 0 1]


 전형적인 이진 분류 모델 설계.
 출력층으로 1개 뉴런 배치, 활성화 함수로 시그모이드 손실함수로 binary_crossentropy 사용.
 총 100회 학습


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

embedding_dim=4
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['acc']
)

model.fit(x_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7065 - acc: 0.0000e+00
Epoch 2/100
1/1 - 0s - loss: 0.7052 - acc: 0.0000e+00
Epoch 3/100
1/1 - 0s - loss: 0.7040 - acc: 0.1429
Epoch 4/100
1/1 - 0s - loss: 0.7027 - acc: 0.1429
Epoch 5/100
1/1 - 0s - loss: 0.7015 - acc: 0.1429
Epoch 6/100
1/1 - 0s - loss: 0.7002 - acc: 0.1429
Epoch 7/100
1/1 - 0s - loss: 0.6990 - acc: 0.1429
Epoch 8/100
1/1 - 0s - loss: 0.6977 - acc: 0.2857
Epoch 9/100
1/1 - 0s - loss: 0.6965 - acc: 0.4286
Epoch 10/100
1/1 - 0s - loss: 0.6953 - acc: 0.4286
Epoch 11/100
1/1 - 0s - loss: 0.6941 - acc: 0.4286
Epoch 12/100
1/1 - 0s - loss: 0.6929 - acc: 0.4286
Epoch 13/100
1/1 - 0s - loss: 0.6917 - acc: 0.4286
Epoch 14/100
1/1 - 0s - loss: 0.6905 - acc: 0.5714
Epoch 15/100
1/1 - 0s - loss: 0.6893 - acc: 0.5714
Epoch 16/100
1/1 - 0s - loss: 0.6881 - acc: 0.5714
Epoch 17/100
1/1 - 0s - loss: 0.6869 - acc: 0.5714
Epoch 18/100
1/1 - 0s - loss: 0.6857 - acc: 0.5714
Epoch 19/100
1/1 - 0s - loss: 0.6845 - acc: 0.5714
Epoch 20/100
1/1 - 0s - loss: 0.

In [18]:
print(model.predict(x_train))

[[0.6384478 ]
 [0.4593276 ]
 [0.4685587 ]
 [0.6036079 ]
 [0.593442  ]
 [0.47361404]
 [0.59137696]]


# 사전 훈련된 GloVe 사용하기

In [ ]:
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall()
zf.close()

In [26]:
from tqdm import tqdm
embedding_dict = dict()
f = open('glove.6B\glove.6B.100d.txt', encoding='utf-8')

for line in tqdm(f):
    word_vector = line.split()
    word = word_vector[0]

    # 100개의 값을 가지는 어레이로 변환
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close()

print('%s개의 Embedding Vector가 있습니다'% len(embedding_dict))

400000it [00:16, 24813.22it/s]

400000개의 Embedding Vector가 있습니다


In [27]:
print(embedding_dict['respectable'])
print('벡터의 차원 수', len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

벡터의 차원수는 100이므로 풀고자 하는 문제와 단어 집합 크기의 행과 100개의 열을 가지는 행렬을 생성. 이 행렬의 값은 모두 0으로 채움

In [28]:
embedding_matrix = np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기(shape) :', np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (16, 100)


기존 데이터와 맵핑된 정수값 확인

In [29]:
print(tokenizer.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


great 값은 2. 사전 훈련된 GloVe 에서의 great 값 확인

In [30]:
print(embedding_dict['great'])

[-0.013786   0.38216    0.53236    0.15261   -0.29694   -0.20558
 -0.41846   -0.58437   -0.77355   -0.87866   -0.37858   -0.18516
 -0.128     -0.20584   -0.22925   -0.42599    0.3725     0.26077
 -1.0702     0.62916   -0.091469   0.70348   -0.4973    -0.77691
  0.66045    0.09465   -0.44893    0.018917   0.33146   -0.35022
 -0.35789    0.030313   0.22253   -0.23236   -0.19719   -0.0053125
 -0.25848    0.58081   -0.10705   -0.17845   -0.16206    0.087086
  0.63029   -0.76649    0.51619    0.14073    1.019     -0.43136
  0.46138   -0.43585   -0.47568    0.19226    0.36065    0.78987
  0.088945  -2.7814    -0.15366    0.01015    1.1798     0.15168
 -0.050112   1.2626    -0.77527    0.36031    0.95761   -0.11385
  0.28035   -0.02591    0.31246   -0.15424    0.3778    -0.13599
  0.2946    -0.31579    0.42943    0.086969   0.019169  -0.27242
 -0.31696    0.37327    0.61997    0.13889    0.17188    0.30363
 -1.2776     0.044423  -0.52736   -0.88536   -0.19428   -0.61947
 -0.10146   -0.26301  

단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑 후, 'great'의 벡터 값이 의도한 인덱스의 위치에 삽입 되었는지 확인

In [35]:
for word, index in tokenizer.word_index.items():
    vector_value = embedding_dict.get(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

값 확인

In [37]:
embedding_matrix[2]

array([-0.013786  ,  0.38216001,  0.53236002,  0.15261   , -0.29694   ,
       -0.20558   , -0.41846001, -0.58437002, -0.77354997, -0.87866002,
       -0.37858   , -0.18516   , -0.12800001, -0.20584001, -0.22925   ,
       -0.42598999,  0.3725    ,  0.26076999, -1.07019997,  0.62915999,
       -0.091469  ,  0.70348001, -0.4973    , -0.77691001,  0.66044998,
        0.09465   , -0.44893   ,  0.018917  ,  0.33146   , -0.35021999,
       -0.35789001,  0.030313  ,  0.22253001, -0.23236001, -0.19719   ,
       -0.0053125 , -0.25848001,  0.58081001, -0.10705   , -0.17845   ,
       -0.16205999,  0.087086  ,  0.63028997, -0.76648998,  0.51618999,
        0.14072999,  1.01900005, -0.43136001,  0.46138   , -0.43584999,
       -0.47567999,  0.19226   ,  0.36065   ,  0.78987002,  0.088945  ,
       -2.78139997, -0.15366   ,  0.01015   ,  1.17980003,  0.15167999,
       -0.050112  ,  1.26259995, -0.77526999,  0.36030999,  0.95761001,
       -0.11385   ,  0.28035   , -0.02591   ,  0.31246001, -0.15

GloVe 에서의 'great 벡터값과 일치함을 보임

Embedding layer 에서 embedding_matrix를 초기값으로 설정하면

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

output_dim = 100

model = Sequential()
e = Embedding(vocab_size, output_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)
# 가중치 초기값 embedding_matrix
# input_length 는 입력 시퀀스의 길이
# trainable 은 사전 훈련된 워드 임베딩을 사용했으므로 추가 훈련을 하지 않는 다는 뜻
model.add(e)
model.add(Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['acc']
)

model.fit(x_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7427 - acc: 0.5357
Epoch 2/100
1/1 - 0s - loss: 0.7390 - acc: 0.5000
Epoch 3/100
1/1 - 0s - loss: 0.7355 - acc: 0.5000
Epoch 4/100
1/1 - 0s - loss: 0.7319 - acc: 0.5000
Epoch 5/100
1/1 - 0s - loss: 0.7284 - acc: 0.5000
Epoch 6/100
1/1 - 0s - loss: 0.7250 - acc: 0.5357
Epoch 7/100
1/1 - 0s - loss: 0.7216 - acc: 0.5357
Epoch 8/100
1/1 - 0s - loss: 0.7183 - acc: 0.5357
Epoch 9/100
1/1 - 0s - loss: 0.7150 - acc: 0.5357
Epoch 10/100
1/1 - 0s - loss: 0.7118 - acc: 0.5357
Epoch 11/100
1/1 - 0s - loss: 0.7086 - acc: 0.5357
Epoch 12/100
1/1 - 0s - loss: 0.7055 - acc: 0.5357
Epoch 13/100
1/1 - 0s - loss: 0.7024 - acc: 0.5357
Epoch 14/100
1/1 - 0s - loss: 0.6993 - acc: 0.5357
Epoch 15/100
1/1 - 0s - loss: 0.6963 - acc: 0.5357
Epoch 16/100
1/1 - 0s - loss: 0.6933 - acc: 0.5357
Epoch 17/100
1/1 - 0s - loss: 0.6904 - acc: 0.5357
Epoch 18/100
1/1 - 0s - loss: 0.6875 - acc: 0.5714
Epoch 19/100
1/1 - 0s - loss: 0.6846 - acc: 0.5714
Epoch 20/100
1/1 - 0s - loss: 0.6818 - a

# 사전 훈련된 Word2Vec 사용하기

In [ ]:
import gensim

urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", \
                           filename="GoogleNews-vectors-negative300.bin.gz")
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

print('모델의 크기(shape) :',word2vec_model.vectors.shape)

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
print('임베딩 행렬의 크기(shape) :', np.shape(embedding_matrix))

특정 단어의 임베딩 벡터를 리턴 받을 수 있는 get_vector 함수 구현

In [ ]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

단어 집합으로부터 단어를 1개씩 호출하여 word2vec_model에 해당 단어의 임베딩 벡터값이 존재하는지 확인.

만약 None이 아니라면 존재한다는 의미이므로 임베딩 행렬에 해당 단어의 인덱스 위치의 행에 임베딩 벡터의 값을 저장

In [ ]:
for word, index in tokenizer.word_index.items():
    vector_value = get_vector(word)
    if  vector_value is not None:
        embedding_matrix[index] = vector_value

모델 설계

In [ ]:
from tensorflow.keras.layers import Input
model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)
model.fit(x_train, y_train, epochs=100, verbose=2)